In [284]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
import time
import csv
import re
from bs4 import BeautifulSoup
import requests

In [285]:
# csv파일 생성 및 쓰기
f = open("albamon_data2.csv", "w")
wrt = csv.writer(f)
wrt.writerow(["근무지역", "기업명", "공고 제목 및 내용", "근무시간", "급여", "공고 등록일", "공고글URL","성별", "연령", "모집직종", "급여", "근무기간", "근무요일", "근무시간", "지원마감일", "근무지주소", "지원방법", "담당자"])

93

In [286]:
# 자동화된 Edge 창 실행
edge_driver = EdgeChromiumDriverManager().install()
service = Service(edge_driver)
driver = webdriver.Edge(service=service)

In [287]:
# 알바몬 웹페이지 접속(광진구, 알바에 해당하는 데이터만 필터링)
url = "https://www.albamon.com/jobs/total?size=50&page=1&condition=%7B%22condition%22%3A%7B%22areas%22%3A%5B%7B%22si%22%3A%22I000%22%2C%22gu%22%3A%22I060%22%2C%22dong%22%3A%22%22%2C%22name%22%3A%22%EA%B4%91%EC%A7%84%EA%B5%AC+%EC%A0%84%EC%B2%B4%22%7D%5D%2C%22similarDongJoin%22%3Afalse%2C%22parts%22%3A%5B%5D%2C%22workPeriodTypes%22%3A%5B%5D%2C%22workWeekTypes%22%3A%5B%5D%2C%22workDayTypes%22%3A%5B%5D%2C%22workTimeTypes%22%3A%5B%5D%2C%22excludeNegoAge%22%3Afalse%2C%22employmentTypes%22%3A%5B%22ALBA%22%5D%2C%22excludeBar%22%3Atrue%7D%2C%22extensionCondition%22%3A%7B%22area%22%3A%7B%7D%2C%22brand%22%3A%7B%7D%2C%22franchise%22%3A%7B%7D%2C%22franchiseStore%22%3A%7B%7D%2C%22callCenter%22%3A%7B%7D%2C%22guerrilla%22%3A%7B%7D%2C%22map%22%3A%7B%22radius%22%3A120%2C%22zoom%22%3A0%7D%2C%22miniJob%22%3A%7B%7D%2C%22ongoing%22%3A%7B%7D%2C%22part%22%3A%7B%7D%2C%22pay%22%3A%7B%7D%2C%22preference%22%3A%7B%7D%2C%22recent%22%3A%7B%7D%2C%22scrap%22%3A%7B%7D%2C%22search%22%3A%7B%22disableExceptedConditions%22%3A%5B%5D%2C%22suitBannerNo%22%3A0%7D%2C%22season%22%3A%7B%7D%2C%22senior%22%3A%7B%7D%2C%22shortTerm%22%3A%7B%7D%2C%22specUp%22%3A%7B%7D%2C%22subway%22%3A%7B%7D%2C%22suit%22%3A%7B%7D%2C%22teenager%22%3A%7B%7D%2C%22town%22%3A%7B%22similarDongJoin%22%3Afalse%7D%2C%22trust%22%3A%7B%7D%2C%22university%22%3A%7B%7D%2C%22welfare%22%3A%7B%7D%7D%7D"
driver.get(url)
time.sleep(2)

In [288]:
# 선택된 창이 꼬이지 않도록 초기화
driver.switch_to.window(driver.window_handles[0])

In [289]:
# 검색 화면 공고 리스트
alba_list = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[1]/main/div[2]/div[1]/div/div[2]/div[2]/ul")

In [290]:
rows = []
for alba in alba_list:
    rows.extend(alba.find_elements(By.TAG_NAME, "li"))
print(len(rows))

50


In [291]:
for row in rows:
    
    # 리스트목록 - 지역, 공고제목, 기업명
    location = row.find_element(By.CLASS_NAME, "list-item-recruit__contents--keyword-area").text
    title = row.find_element(By.CLASS_NAME, 'list-item-recruit__recruit-title').text
    company = row.find_element(By.CLASS_NAME, 'ListItemRecruit_list-item-recruit__company-name__bbljH').text
    
    # 공고URL
    link = row.find_element(By.CLASS_NAME, "list-item-recruitgoogle-analytics")
    link_url = link.get_attribute("href")

    # 공고URL에 접속
    driver.execute_script("window.open(arguments[0]);", link_url)
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
    
    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")

    ## 공고등록일
    reg_date = driver.find_element(By.CLASS_NAME, "job-detail__registered").text
    
    ## 모집조건 리스트 선택 - 모집마감, 성별, 연령
    applyconditions = soup.find("div", class_="detail-recruit-work-apply__container")
    AC = {}
    # 각 dl 태그에서 변수명과 변수값 가져오기
    dl_tags = applyconditions.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        AC[dt] = dd
        
    ## 근무주소
    address = driver.find_element(By.CLASS_NAME, "detail-recruit-area__address").text
        
    ## 근무조건 리스트 선택 - 급여(형태 및 금액), 근무기간, 근무요일, 근무시간, 업직종
    workconditions = soup.find("div", class_="detail-recruit-conditions__container")
    WC = {}
    # 각 dl 태그에서 변수명과 변수값 가져오기
    dl_tags = workconditions.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        WC[dt] = dd
    
   

    # 지원방법 추출
    apply_method = soup.find("div", class_="detail-recruit-apply__container")
    apply = []
    if apply_method is not None:
        buttons = apply_method.find_all("button")
        for button in buttons:
            spans = button.find_all("span")
            for span in spans:
                if span is not None:
                    apply.append(span.text.strip())

    # 채용 담당자 연락처 추출
    manager_contact = {}
    manager_frame = soup.find("div", class_="job-detail__iframe")
    if manager_frame is not None:
        iframe_url = manager_frame.find("iframe")["src"]
        iframe_response = requests.get("https://www.albamon.com"+iframe_url)
        iframe_soup = BeautifulSoup(iframe_response.text, "html.parser")
        manager_info = iframe_soup.find("div", class_="detail-recruit-manager__container")
        if manager_info is not None:
            dl_tags = manager_info.find_all("dl")
            for dl in dl_tags:
                dt_element = dl.find("dt")
                dd_element = dl.find("dd")
                if dt_element is not None and dd_element is not None:
                    dt = dt_element.text.strip()
                    dd = dd_element.text.strip()
                    manager_contact[dt] = dd

        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    wrt.writerow([location, company, title, WC["근무시간"], WC["급여"], reg_date, link_url, AC["성별"], AC["연령"],WC["업직종"], WC["급여"], WC["근무기간"], WC["근무요일"], WC["근무시간"], AC["모집마감"], address, apply, manager_contact])

In [292]:
#공고 창 및 파일 닫기
driver.close()
f.close()